## Hydraulic head data 

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import flopy
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings

# Ignore all DeprecationWarnings (including distutils)
warnings.filterwarnings("ignore", category=DeprecationWarning)

mfnam = "Example.nam"
model_ws = "data_path"
exe_path = "...\mfusg-1.4.exe"
m = flopy.mfusg.MfUsg.load(mfnam, exe_name=exe_path, model_ws=model_ws)
# Load the head file
headobj = flopy.utils.HeadUFile("...\Example.hds")
head = headobj.get_data(kstpkper=(0,10))

## MODFLOW USG Hydraulic Head Data Visualization

In [ ]:
"""
Modflow USG Hydraulic Head Data Visualization
============================================

A comprehensive Python script for visualizing Modflow USG (Unstructured Grid) 
hydraulic head data with proper .gsf file parsing for grid geometry.

Requirements:
- numpy
- matplotlib
- pandas
- flopy (for Modflow file reading)
- scipy (for interpolation)
- plotly (optional, for interactive plots)

Make sure you have model.gsf and model.hds files in the same directory,
or modify the file paths in the main() function.
"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import pandas as pd
from pathlib import Path
import struct
import warnings
warnings.filterwarnings('ignore')

try:
    import flopy
    FLOPY_AVAILABLE = True
except ImportError:
    FLOPY_AVAILABLE = False
    print("Warning: flopy not available. Some features may not work.")

try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("Info: plotly not available. Interactive plots disabled.")

class ModflowUSGVisualizer:
    """
    Main class for visualizing Modflow USG hydraulic head data with .gsf file support
    """
    
    def __init__(self, model_dir=None):
        """
        Initialize the visualizer
        
        Parameters:
        -----------
        model_dir : str, optional
            Path to the directory containing Modflow USG files
        """
        self.model_dir = Path(model_dir) if model_dir else Path.cwd()
        self.heads = None
        self.node_coords = None
        self.cell_vertices = None
        self.cell_nodes = None
        self.node_to_cell = None
        self.layer_info = None
        self.grid_info = None
        self.stats = {}
        
    def read_gsf_file(self, gsf_file_path):
        """
        Read Grid Specification File (.gsf) to extract unstructured grid geometry
        
        Parameters:
        -----------
        gsf_file_path : str
            Path to the .gsf file
        """
        gsf_file_path = Path(gsf_file_path)
        
        if not gsf_file_path.exists():
            raise FileNotFoundError(f"GSF file not found: {gsf_file_path}")
        
        print(f"Reading GSF file: {gsf_file_path}")
        
        try:
            # Try using flopy first if available
            if FLOPY_AVAILABLE:
                try:
                    # Read using flopy's USG grid reader
                    self._read_gsf_with_flopy(gsf_file_path)
                    return True
                except Exception as e:
                    print(f"Flopy GSF reading failed: {e}")
                    print("Attempting manual GSF parsing...")
            
            
        except Exception as e:
            print(f"Error reading GSF file: {e}")
            return False
    
    def _read_gsf_with_flopy(self, gsf_file_path):
        """Read GSF file using flopy"""
        try:
            # Try newer flopy API first
            from flopy.discretization import UnstructuredGrid
            usg_grid = UnstructuredGrid.from_gridspec(str(gsf_file_path))
        except (ImportError, AttributeError):
            try:
                # Try older flopy API
                usg_grid = flopy.utils.UnstructuredGrid.from_gridspec(str(gsf_file_path))
            except AttributeError:
                # Try alternative flopy method
                import flopy.utils.gridutil as gridutil
                usg_grid = gridutil.read_gridspec(str(gsf_file_path))
        
        # Extract grid information
        if hasattr(usg_grid, 'xcellcenters') and hasattr(usg_grid, 'ycellcenters'):
            self.node_coords = np.column_stack((usg_grid.xcellcenters, usg_grid.ycellcenters))
        elif hasattr(usg_grid, 'cell_centers'):
            self.node_coords = usg_grid.cell_centers[:, :2]
        
        if hasattr(usg_grid, 'vertices'):
            self.cell_vertices = usg_grid.vertices
        if hasattr(usg_grid, 'iverts'):
            self.cell_nodes = usg_grid.iverts
        
        # Store grid info
        self.grid_info = {
            'nnodes': getattr(usg_grid, 'nnodes', len(self.node_coords) if self.node_coords is not None else 0),
            'ncpl': getattr(usg_grid, 'ncpl', None),
            'nlay': getattr(usg_grid, 'nlay', 1),
            'nvert': getattr(usg_grid, 'nvert', None)
        }
        
        print(f"Grid loaded via flopy: {self.grid_info['nnodes']} nodes, {self.grid_info['nlay']} layers")
    
   
            
    def read_usg_head_file(self, head_file_path):
        """
        Read hydraulic head data from USG head file
        
        Parameters:
        -----------
        head_file_path : str
            Path to the head file (.hds or .hed)
        """
        head_file_path = Path(head_file_path)
        
        if not head_file_path.exists():
            raise FileNotFoundError(f"Head file not found: {head_file_path}")
        
        if FLOPY_AVAILABLE:
            try:
                # Try using flopy first
                headobj = flopy.utils.HeadUFile(str(head_file_path))
                times = headobj.get_times()
                heads_data = headobj.get_data(totim=times[-1])  # Get last time step
                
                # Handle different data formats
                if isinstance(heads_data, list):
                    # If it's a list, it might be layer-by-layer data
                    self.heads = np.concatenate([np.array(layer).flatten() for layer in heads_data])
                elif isinstance(heads_data, np.ndarray):
                    # If it's already an array, flatten if multi-dimensional
                    if heads_data.ndim > 1:
                        self.heads = heads_data.flatten()
                    else:
                        self.heads = heads_data
                else:
                    # Try to convert to array
                    self.heads = np.array(heads_data).flatten()
                
                print(f"Successfully read head data using flopy. Shape: {self.heads.shape}")
                print(f"Time steps available: {len(times)}, using time: {times[-1]}")
                return True
            except Exception as e:
                print(f"Flopy reading failed: {e}")
                print("Attempting binary read...")
        
    
    def calculate_statistics(self):
        """Calculate basic statistics for the head data"""
        if self.heads is None:
            return
        
        # Handle inactive/dry cells (often marked with large negative values)
        active_mask = (self.heads > -1e10) & (self.heads < 1e10) & (~np.isnan(self.heads))
        valid_heads = self.heads[active_mask]
        
        if len(valid_heads) == 0:
            print("No valid head values found!")
            return
        
        self.stats = {
            'min': np.min(valid_heads),
            'max': np.max(valid_heads),
            'mean': np.mean(valid_heads),
            'median': np.median(valid_heads),
            'std': np.std(valid_heads),
            'count': len(valid_heads),
            'inactive_count': len(self.heads) - len(valid_heads),
            'total_nodes': len(self.heads)
        }
        
        print("\nHydraulic Head Statistics:")
        print(f"  Min:    {self.stats['min']:.3f} m")
        print(f"  Max:    {self.stats['max']:.3f} m")
        print(f"  Mean:   {self.stats['mean']:.3f} m")
        print(f"  Median: {self.stats['median']:.3f} m")
        print(f"  Std:    {self.stats['std']:.3f} m")
        print(f"  Active nodes: {self.stats['count']}")
        print(f"  Inactive nodes: {self.stats['inactive_count']}")
        print(f"  Total nodes: {self.stats['total_nodes']}")
    
    def create_unstructured_plot(self, save_path=None, figsize=(15, 10), layer=0):
        """
        Create visualization specifically for unstructured grids
        
        Parameters:
        -----------
        save_path : str, optional
            Path to save the figure
        figsize : tuple
            Figure size (width, height)
        layer : int
            Layer to visualize (for multi-layer models)
        """
        if self.heads is None or self.node_coords is None:
            raise ValueError("Both head data and grid geometry must be loaded")
        
        # Filter active cells
        active_mask = (self.heads > -1e10) & (self.heads < 1e10) & (~np.isnan(self.heads))
        
        fig, axes = plt.subplots(2, 2, figsize=figsize)
        axes = axes.flatten()
        
        x = self.node_coords[:, 0]
        y = self.node_coords[:, 1]
        z = self.heads
        
        x_active = x[active_mask]
        y_active = y[active_mask]
        z_active = z[active_mask]
        
        # 1. Scatter plot with color mapping
        scatter = axes[0].scatter(x_active, y_active, c=z_active, cmap='viridis', 
                                s=20, alpha=0.8, edgecolors='black', linewidths=0.1)
        axes[0].set_title('USG Node Head Values', fontsize=12, fontweight='bold')
        axes[0].set_xlabel('X Coordinate (m)')
        axes[0].set_ylabel('Y Coordinate (m)')
        axes[0].set_aspect('equal')
        plt.colorbar(scatter, ax=axes[0], label='Head (m)')
        
        # 2. Triangulated contour plot
        if len(x_active) > 3:
            try:
                triang = tri.Triangulation(x_active, y_active)
                levels = np.linspace(z_active.min(), z_active.max(), 15)
                cs = axes[1].tricontourf(triang, z_active, levels=levels, cmap='plasma')
                axes[1].tricontour(triang, z_active, levels=levels[::2], colors='white', linewidths=0.5)
                axes[1].set_title('Triangulated Contours', fontsize=12, fontweight='bold')
                axes[1].set_xlabel('X Coordinate (m)')
                axes[1].set_ylabel('Y Coordinate (m)')
                axes[1].set_aspect('equal')
                plt.colorbar(cs, ax=axes[1], label='Head (m)')
            except Exception as e:
                axes[1].text(0.5, 0.5, f'Triangulation failed:\n{str(e)}', 
                           transform=axes[1].transAxes, ha='center', va='center')
                axes[1].set_title('Triangulation Error')
        
        # 3. Cell-based visualization (if cell connectivity available)
        if self.cell_nodes and len(self.cell_nodes) > 0:
            self._plot_cell_patches(axes[2], active_mask)
        else:
            # Fallback: Voronoi-style plot
            scatter2 = axes[2].scatter(x_active, y_active, c=z_active, cmap='coolwarm', 
                                     s=30, alpha=0.7)
            axes[2].set_title('Node Values (No Cell Data)', fontsize=12, fontweight='bold')
            axes[2].set_xlabel('X Coordinate (m)')
            axes[2].set_ylabel('Y Coordinate (m)')
            axes[2].set_aspect('equal')
            plt.colorbar(scatter2, ax=axes[2], label='Head (m)')
        
        # 4. Grid structure
        axes[3].plot(x, y, 'k.', markersize=2, alpha=0.5, label='All nodes')
        axes[3].plot(x_active, y_active, 'r.', markersize=3, label='Active nodes')
        axes[3].set_title(f'Grid Structure ({len(x)} nodes)', fontsize=12, fontweight='bold')
        axes[3].set_xlabel('X Coordinate (m)')
        axes[3].set_ylabel('Y Coordinate (m)')
        axes[3].set_aspect('equal')
        axes[3].legend()
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"Unstructured grid plot saved to: {save_path}")
        
        plt.show()
    
    def _plot_cell_patches(self, ax, active_mask):
        """Plot cells as patches using connectivity information"""
        patches = []
        colors = []
        
        for i, cell_vertices in enumerate(self.cell_nodes):
            if i < len(self.heads) and active_mask[i]:
                # Get vertex coordinates
                try:
                    vertex_coords = self.node_coords[cell_vertices]
                    polygon = Polygon(vertex_coords, closed=True)
                    patches.append(polygon)
                    colors.append(self.heads[i])
                except IndexError:
                    continue
        
        if patches:
            collection = PatchCollection(patches, cmap='viridis', alpha=0.8)
            collection.set_array(np.array(colors))
            ax.add_collection(collection)
            
            # Set axis limits
            ax.set_xlim(self.node_coords[:, 0].min(), self.node_coords[:, 0].max())
            ax.set_ylim(self.node_coords[:, 1].min(), self.node_coords[:, 1].max())
            ax.set_title('Cell-based Visualization', fontsize=12, fontweight='bold')
            ax.set_xlabel('X Coordinate (m)')
            ax.set_ylabel('Y Coordinate (m)')
            ax.set_aspect('equal')
            
            plt.colorbar(collection, ax=ax, label='Head (m)')
    
    def create_3d_unstructured_plot(self, save_path=None, figsize=(12, 9)):
        """
        Create a 3D visualization for unstructured grid
        """
        if self.heads is None or self.node_coords is None:
            raise ValueError("Both head data and grid geometry must be loaded")
        
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111, projection='3d')
        
        # Filter active cells
        active_mask = (self.heads > -1e10) & (self.heads < 1e10) & (~np.isnan(self.heads))
        
        x = self.node_coords[active_mask, 0]
        y = self.node_coords[active_mask, 1]
        z = self.heads[active_mask]
        
        # Create 3D scatter plot
        scatter = ax.scatter(x, y, z, c=z, cmap='coolwarm', s=15, alpha=0.7)
        
        ax.set_xlabel('X Coordinate (m)')
        ax.set_ylabel('Y Coordinate (m)')
        ax.set_zlabel('Hydraulic Head (m)')
        ax.set_title('3D Unstructured Grid Head Distribution', fontsize=14, fontweight='bold')
        
        plt.colorbar(scatter, ax=ax, label='Hydraulic Head (m)', shrink=0.6)
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"3D unstructured plot saved to: {save_path}")
        
        plt.show()
    
    def create_interactive_unstructured_plot(self, save_path=None):
    #     """
    #     Create an interactive plot for unstructured grid using plotly
    #     """
        if not PLOTLY_AVAILABLE:
            print("Plotly not available. Install plotly for interactive plots.")
            return
        
        if self.heads is None or self.node_coords is None:
            raise ValueError("Both head data and grid geometry must be loaded")
        
        # Filter active cells
        active_mask = (self.heads > -1e10) & (self.heads < 1e10) & (~np.isnan(self.heads))
        
        x = self.node_coords[active_mask, 0]
        y = self.node_coords[active_mask, 1]
        z = self.heads[active_mask]
        
        fig = go.Figure(data=go.Scatter(
            x=x,
            y=y,
            mode='markers',
            marker=dict(
                size=6,
                color=z,
                colorscale='Viridis',
                colorbar=dict(title="Hydraulic Head (m)"),
                line=dict(width=0.5, color='white')
            ),
            text=[f'Node: {i}<br>Head: {z[i]:.3f} m<br>X: {x[i]:.1f}<br>Y: {y[i]:.1f}' 
                  for i in range(len(x))],
            hovertemplate='%{text}<extra></extra>',
            name='Active Nodes'
        ))
        
        fig.update_layout(
            title='Interactive USG Hydraulic Head Distribution',
            xaxis_title='X Coordinate (m)',
            yaxis_title='Y Coordinate (m)',
            width=900,
            height=700,
            hovermode='closest'
        )
        
        # Add grid info annotation
        fig.add_annotation(
            text=f"Total nodes: {self.grid_info.get('nnodes', 'Unknown')}<br>"
                 f"Active nodes: {len(x)}<br>"
                 f"Head range: {z.min():.2f} - {z.max():.2f} m",
            xref="paper", yref="paper",
            x=0.02, y=0.98, xanchor="left", yanchor="top",
            showarrow=False,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="gray",
            borderwidth=1
        )
        
        if save_path:
            fig.write_html(save_path)
            print(f"Interactive USG plot saved to: {save_path}")
        
        fig.show()
    
    def export_usg_data(self, export_path, format='csv'):
        """
        Export USG data to various formats
        
        Parameters:
        -----------
        export_path : str
            Path for the exported file
        format : str
            Export format ('csv', 'xlsx', 'txt')
        """
        if self.heads is None or self.node_coords is None:
            raise ValueError("Both head data and grid geometry must be loaded")
        
        # Create comprehensive DataFrame
        active_mask = (self.heads > -1e10) & (self.heads < 1e10) & (~np.isnan(self.heads))
        
        df = pd.DataFrame({
            'node_id': range(len(self.heads)),
            'x_coord': self.node_coords[:, 0],
            'y_coord': self.node_coords[:, 1],
            'hydraulic_head': self.heads,
            'is_active': active_mask
        })
        
        # Add layer information if available
        if self.grid_info and 'nlay' in self.grid_info and self.grid_info['nlay'] > 1:
            nodes_per_layer = len(self.heads) // self.grid_info['nlay']
            df['layer'] = df['node_id'] // nodes_per_layer + 1
        
        if format.lower() == 'csv':
            df.to_csv(export_path, index=False)
        elif format.lower() == 'xlsx':
            df.to_excel(export_path, index=False)
        elif format.lower() == 'txt':
            df.to_csv(export_path, sep='\t', index=False)
        
        print(f"USG data exported to: {export_path}")

def main():
    """
    Main function demonstrating usage with USG files
    """
    print("Modflow USG Hydraulic Head Visualizer")
    print("=" * 40)
    
    # Initialize visualizer
    viz = ModflowUSGVisualizer()
    
    # Example usage - adjust paths as needed
    gsf_file = "...\Example.gsf"      # Grid specification file
    head_file = "...\Example.hds"     # Head file
    
    try:
        # Load grid geometry from GSF file
        print("Loading USG grid geometry from GSF file...")
        if not viz.read_gsf_file(gsf_file):
            print("Failed to load GSF file. Please check file path and format.")
            return
        
        # Load head data
        print("Loading head data...")
        if not viz.read_usg_head_file(head_file):
            print("Failed to load head data. Please check file path and format.")
            return
        
        # Calculate statistics
        viz.calculate_statistics()
        
        # Create visualizations
        print("\nCreating USG visualizations...")
        
        # Unstructured grid specific plots
        viz.create_unstructured_plot(save_path="usg_head_analysis.png")
        
        # 3D visualization
        viz.create_3d_unstructured_plot(save_path="usg_head_3d.png")
        
        # Interactive plot (if plotly available)
        if PLOTLY_AVAILABLE:
            viz.create_interactive_unstructured_plot(save_path="usg_head_interactive.html")
        
        # Export data
        viz.export_usg_data("usg_head_data_export.csv", format='csv')
        
        print("\nUSG visualization complete!")
        
    except Exception as e:
        print(f"Error: {e}")
        print("Please check your USG file paths and data format.")
        print("\nExpected files:")
        print("- model.gsf (Grid Specification File)")
        print("- model.hds (Head file)")

if __name__ == "__main__":
    main()